In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

/opt/conda/envs/default/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# 语料去重 （可运行）

# 读取CSV文件
# file_path = 'mangoNews_Example.csv'  # 测试版数据集（规模小5m）
file_path = 'datasets/mangoNews.csv'          # 完整版数据集（13g） 需要分块读
# file_path = 'datasets/mangoNews_Example_100000.csv'          # 100000行数据集（2.2g）
# file_path = 'mangoNews_Example_10000.csv'          # 10000行数据集（190m)

data = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

# 统计去重前的总语料数
total_records_before = len(data)

# 根据多个列进行去重，并保留重复行
# duplicates = data[data.duplicated(subset=["website_id", "request_url","response_url","category1", "category2", "title", "body","pub_time"], keep=False)]

# 输出重复行
# print("Duplicate records:")
# print(duplicates)

# 对 'body'列内容都重复的语料进行去重处理
# data.drop_duplicates(subset=['body'], inplace=True)
# data.drop_duplicates(subset=["category1", "category2", "title", "body"], inplace=True) # 100025  99352
# data.drop_duplicates(subset=["website_id", "request_url", "category1", "category2", "title", "body"], inplace=True) # 100025  99534
# data.drop_duplicates(subset=["website_id", "request_url","response_url","category1", "category2", "title", "body"], inplace=True) # 100025  99534
# data.drop_duplicates(subset=["website_id", "request_url","response_url","category1", "category2", "title", "body","pub_time"], inplace=True) # 100025  99829
# data.drop_duplicates(subset=["website_id", "request_url","response_url","category1", "category2", "title", "body","pub_time","cole_time"], inplace=True) # 100025  100019
# data.drop_duplicates(subset=["website_id", "request_url","response_url","category1", "category2", "title", "body","pub_time","cole_time","images"], inplace=True) # 100025  100022
# 不同时间爬到同一篇（除爬取时间和id外其他相同）
data.drop_duplicates(subset=["website_id", "request_url","response_url","category1", "category2", "title", "abstract","body","pub_time","images"], inplace=True) # 100025  99862

# data.drop_duplicates(subset=["md5"], inplace=True) # 100025  99474


# 统计去重后的总语料数
total_records_after = len(data)

data.to_csv('datasets_FIX/FIX_deduplicated_mangoNews.csv', index=False)


# 打印统计结果
print(f"Total records before deduplication: {total_records_before}")
print(f"Total records after deduplication: {total_records_after}")
# 10000：10007 9969
# 100000:100025 98912

# new tragedy： 1663846 1661972 !!! new

Total records before deduplication: 1663846
Total records after deduplication: 1661972


In [ ]:
# 语料去重 大语料分块处理 （别运行这个，虽然有整体去重的操作，但对于完整的数据集会爆）
import pandas as pd
import os

# 读取CSV文件，使用chunksize进行分块读取
file_path = 'mangoNews.csv'          # 完整版数据集（13g） 需要分块读
# file_path = 'mangoNews_Example_100000.csv'          # 100000行数据集（2.2g）
# file_path = 'mangoNews_Example_10000.csv'          # 10000行数据集（190m)

chunk_size = 100000  # 根据实际情况调整分块大小
temp_files = []

# 初始化空的DataFrame，用于存储全局去重后的数据
data_deduped = pd.DataFrame(columns=['body'])

total_records_before2 = 0
total_records_after2 = 0

# 遍历分块数据
for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)):
    # 统计当前分块的记录数并累加到总记录数
    print(i,len(chunk))
    total_records_before2 += len(chunk)
    
    # 对当前分块数据进行去重操作
    chunk.drop_duplicates(subset=['body'], inplace=True)

    # 将当前分块数据写入临时文件
    temp_file_path = f'temp_chunk_{i}.csv'
    chunk.to_csv(temp_file_path, index=False)
    temp_files.append(temp_file_path)
    
    

# 读取并合并所有临时文件的数据
for temp_file in temp_files:
    chunk_deduped = pd.read_csv(temp_file)
    data_deduped = pd.concat([data_deduped, chunk_deduped], ignore_index=True)

# 对合并后的 DataFrame 再次进行全局的去重操作
data_deduped.drop_duplicates(subset=['body'], inplace=True)

# 删除临时文件
for temp_file in temp_files:
    os.remove(temp_file)

# 统计去重前后的总语料数
# total_records_before = len(data)
total_records_after2 = len(data_deduped)

# 打印统计结果
print(f"Total records before deduplication: {total_records_before2}")
print(f"Total records after deduplication: {total_records_after2}")


0 100000
1 100000
2 100000
3 100000
4 100000
5 100000
6 100000
7 100000
8 100000
9 100000
10 100000
11 100000
12 100000
13 100000


/tmp/ipykernel_5413/1571218463.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):


14 100000
15 100000
16 63846


/tmp/ipykernel_5413/1571218463.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_deduped = pd.read_csv(temp_file)
/tmp/ipykernel_5413/1571218463.py:37: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_deduped = pd.read_csv(temp_file)
/tmp/ipykernel_5413/1571218463.py:37: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_deduped = pd.read_csv(temp_file)


In [3]:
# 爆存 退而求其次 （可运行）
import pandas as pd

# 读取CSV文件，使用chunksize进行分块读取
file_path = 'mangoNews.csv'          # 完整版数据集（13g） 需要分块读
# file_path = 'mangoNews_Example_100000.csv'          # 100000行数据集（2.2g）
# file_path = 'mangoNews_Example_10000.csv'          # 10000行数据集（190m)

chunk_size = 300000  # 根据实际情况调整分块大小
data_chunks = pd.read_csv(file_path, chunksize=chunk_size, low_memory=False)

# 初始化空的 DataFrame，用于存储去重后的数据
deduplicated_data = pd.DataFrame(columns=['body'])

total_records_before3 = 0
total_records_after3 = 0

# # 将去重后的数据写入新的CSV文件
deduplicated_file_path = 'deduplicated_mangoNews.csv'  # 新CSV文件路径

# 遍历分块数据
for i, chunk in enumerate(data_chunks):
    # 统计当前分块的记录数
    total_records_before3 += len(chunk)
    print(i,len(chunk))

    # 对 'body'列内容都重复的语料进行去重处理
    chunk.drop_duplicates(subset=['body'], inplace=True)

    # 统计去重后的总语料数
    total_records_after3 += len(chunk)
    
    # 将去重后的数据追加到同一个 CSV 文件中
    mode = 'w' if i == 0 else 'a'  # 第一个分块使用 'w' 模式，后续分块使用 'a' 模式追加数据
    header = False if i != 0 else True  # 只在第一个分块写入时写入表头
    
    chunk.to_csv(deduplicated_file_path, index=False, mode=mode, header=header)



# deduplicated_data.to_csv(deduplicated_file_path, index=False)

# print("test")
# print(deduplicated_data.shape[0])
# print(len(deduplicated_data))

# 打印统计结果
print(f"Total records before deduplication: {total_records_before3}")
print(f"Total records after deduplication: {total_records_after3}")


0 300000
1 300000
2 300000
3 300000
4 300000
5 163846
Total records before deduplication: 1663846
Total records after deduplication: 1347969


In [26]:
# 'mangoNews_10000.csv'和'mangoNews_100000.csv'中类别统计并保存
# 读取CSV文件
# file_path = 'mangoNews_Example.csv'  # 测试版数据集（规模小5m）
# file_path = 'datasets/mangoNews.csv'          # 完整版数据集（13g） 需要分块读
# file_path = 'mangoNews_Example_100000.csv'          # 100000行数据集（2.2g）
# file_path = 'mangoNews_Example_10000.csv'          # 10000行数据集（190m)

file_path = 'datasets_FIX/FIX_deduplicated_mangoNews.csv'          # 完整版数据集（13g） 需要分块读

data = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

# Select relevant columns
data = data[['body', 'category1']]

# 统计 'category1' 列中每种类别的个数
category_counts = data['category1'].value_counts()

# # 设置显示选项，完整输出结果
# pd.set_option('display.max_rows', None)
# print("Category Counts:")
# print(category_counts)
# # 恢复默认显示选项
# pd.reset_option('display.max_rows')

# 将结果保存到CSV文件
result_df = pd.DataFrame({'category1': category_counts.index, 'num': category_counts.values})
# result_df.to_csv('res_files_FIX/mangoNews_category_counts_all.csv', index=False)
result_df.to_csv('res_files_FIX/FIX_deduplicated_mangoNews_category_counts_all.csv', index=False)


# 打印结果
print("Category Counts:")
print(result_df)


Category Counts:
              category1     num
0                 সংবাদ  414224
1              বাংলাদেশ  126016
2         আজকের পত্রিকা   94927
3           আন্তর্জাতিক   75581
4                 জাতীয়   73246
..                  ...     ...
789                   এ       1
790               ভারত        1
791            আবু রুশদ       1
792         জগদীশ গুপ্ত       1
793  সুকুমার ভট্টাচার্য       1

[794 rows x 2 columns]


In [9]:
# 'mangoNews.csv'中类别统计并保存
# 读取CSV文件
# file_path = 'mangoNews_Example.csv'  # 测试版数据集（规模小5m）
file_path = 'mangoNews.csv'          # 完整版数据集（13g） 需要分块读
# file_path = 'mangoNews_Example_100000.csv'          # 100000行数据集（2.2g）
# file_path = 'mangoNews_Example_10000.csv'          # 10000行数据集（190m)

# 分块读取：
# 读取CSV文件，使用chunksize进行分块读取
# file_path = 'your_dataset.csv'  # 替换为你的数据集路径
chunk_size = 100000  # 根据实际情况调整分块大小
data_chunks = pd.read_csv(file_path, usecols=['body', 'category1'], chunksize=chunk_size)

# 初始化空的DataFrame，用于存储统计结果
result_df = pd.DataFrame(columns=['category1', 'num'])

# 遍历分块数据
for chunk in data_chunks:
    # 统计 'category1' 列中每种类别的个数
    category_counts = chunk['category1'].value_counts().reset_index()
    category_counts.columns = ['category1', 'num']

    # 合并到总的结果DataFrame
    result_df = pd.concat([result_df, category_counts], ignore_index=True)

# 合并结果中的重复项，得到最终的统计结果
final_result = result_df.groupby('category1')['num'].sum().reset_index()

# 按照 'num' 列降序排序
final_result = final_result.sort_values(by='num', ascending=False)

# 将最终结果保存到CSV文件
final_result.to_csv('category_counts_all.csv', index=False)

# 打印结果
# 设置显示选项，完整输出结果
# pd.set_option('display.max_rows', None)
print("Category Counts (Sorted):")
print(final_result)
# 恢复默认显示选项
# pd.reset_option('display.max_rows')

Category Counts (Sorted):
          category1     num
701           সংবাদ  415412
493        বাংলাদেশ  126064
166   আজকের পত্রিকা   94927
173     আন্তর্জাতিক   75581
319           জাতীয়   73246
..              ...     ...
555     মঞ্জু সরকার       1
153     অরূপরতন ঘোষ       1
152  অরুণকুমার দত্ত       1
561        মনোজ বসু       1
793            ২০২০       1

[794 rows x 2 columns]


In [ ]:
# 将类别列转换为整数标签  注意是data['category1']
label_to_id = {label: idx for idx, label in enumerate(data['category1'].unique())}
print(label_to_id)
data['label'] = data['category1'].map(label_to_id)
num_classes = len(label_to_id)
print(num_classes)